# dataset 로드

In [55]:
import torchvision
import torch
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor
data_path = '/Users/minsu/Desktop/archive/Pytorch-Master/8장-컨볼루션을-활용한-일반화/dataset/'
cifar10 = datasets.CIFAR10(data_path , train = True, download = False, transform=ToTensor())
cifar10_val = datasets.CIFAR10(data_path, train = False, download = False,transform=ToTensor())

In [56]:
label_map = {0:0 , 2:1}
class_names = ['airplane','bird']

cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]


In [57]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(3,16, kernel_size = 3, padding = 1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(16,8,kernel_size = 3, padding = 1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Linear(8*8*8, 32),
    nn.Tanh(),
    nn.Linear(32,2)
)

In [58]:
model

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Tanh()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Tanh()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Linear(in_features=512, out_features=32, bias=True)
  (7): Tanh()
  (8): Linear(in_features=32, out_features=2, bias=True)
)

In [59]:
# 이 모델에서 오류가 나는 이유는 8 채널의 8 * 8 이미지를 512 요소를 가진 1차원 벡터로 차원 정보를 변경해야 하는 부분이다.
# 마지막 nn.MaxPool2d의 출력에 대해 view 를 호출하면 해결 할 수 있지만 nn.Sequential을 사요할 때는 각 모듈의 출력을 명시적으로 볼 수 없어서 불편함이 있다. 

# nn.Modul 서브 클래싱 하기 

- nn.sequential 을 이용한 것이 아닌 custom 모델 구조 만들기

In [60]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size = 3, padding = 1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16,8, kernel_size = 3 , padding = 1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32,2)

    def forward(self,x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8*8*8) # 앞에서의 놓쳤던 차원 정보
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out
    
    

In [61]:
model = Net()
model

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): Tanh()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): Tanh()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (act3): Tanh()
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

## 함수 API 로 대체 시킨 코드

In [62]:
import torch.nn.functional as F
import torch.nn as nn
import torch

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(8*8*8, 32)
        self.fc2 = nn.Linear(32,2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1,8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out
    


In [63]:
model = Net()
model 

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=512, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

In [64]:
from torchvision.transforms import ToTensor

img , _ = cifar10[0]
trans = ToTensor()

img = trans(img)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [65]:
model = Net()
model(img.unsqueeze(0))


tensor([[0.0669, 0.1172]], grad_fn=<AddmmBackward0>)

In [66]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)

            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {} , Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)
            ))


In [67]:
train_loader = torch.utils.data.DataLoader(cifar2 , batch_size = 64, shuffle = True)

model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()

In [68]:
training_loop(
    n_epochs= 100,
    optimizer= optimizer,
    model= model,
    loss_fn= loss_fn,
    train_loader=train_loader,
)

2024-04-29 13:49:55.353044 Epoch 1 , Training loss 0.6828592518332658
2024-04-29 13:50:18.227271 Epoch 10 , Training loss 0.46621412893009795
2024-04-29 13:50:43.598237 Epoch 20 , Training loss 0.36545680738558434
2024-04-29 13:51:09.344358 Epoch 30 , Training loss 0.3268655798617442
2024-04-29 13:51:34.790777 Epoch 40 , Training loss 0.31205012132028104
2024-04-29 13:52:00.333470 Epoch 50 , Training loss 0.2964702324502787
2024-04-29 13:52:25.850126 Epoch 60 , Training loss 0.2818504317550902
2024-04-29 13:52:51.358406 Epoch 70 , Training loss 0.269080149519975
2024-04-29 13:53:16.858218 Epoch 80 , Training loss 0.25243407687184155
2024-04-29 13:53:43.344790 Epoch 90 , Training loss 0.23969836091729485
2024-04-29 13:54:11.069862 Epoch 100 , Training loss 0.22583079266889838


In [69]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size = 64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size = 64, shuffle = False)

In [76]:
def validate(model, tarin_loader, val_loader):
    for name, loader in [('train', train_loader), ('val', val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {} : {:.2f}".format(name, correct / total))

validate(model, train_loader , val_loader)

Accuracy train : 0.90
Accuracy val : 0.89


In [ ]:
## 모델을 저장하고 불러오기

torch.save(model.state_dict(), data_path + 'birds+vs+airbl')

## gpu 활용하기

In [77]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [81]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device = device)
            labels = labels.to(device = device)
            outputs = model(imgs)

            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {} , Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)
            ))


In [82]:
train_loader = torch.utils.data.DataLoader(cifar2 , batch_size = 64, shuffle = True)

model = Net().to(device = device)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()

In [83]:
training_loop(
    n_epochs= 100,
    optimizer= optimizer,
    model= model,
    loss_fn= loss_fn,
    train_loader=train_loader,
)

2024-04-29 14:06:32.675797 Epoch 1 , Training loss 0.6838479877277545
2024-04-29 14:06:38.368610 Epoch 10 , Training loss 0.46247263546961886
2024-04-29 14:06:44.512525 Epoch 20 , Training loss 0.3670584877395326
2024-04-29 14:06:50.621817 Epoch 30 , Training loss 0.3299683908558196
2024-04-29 14:06:56.722389 Epoch 40 , Training loss 0.310187829717709
2024-04-29 14:07:02.881702 Epoch 50 , Training loss 0.29644258386769873
2024-04-29 14:07:08.974945 Epoch 60 , Training loss 0.2810719429877154
2024-04-29 14:07:15.096294 Epoch 70 , Training loss 0.26866504170332745
2024-04-29 14:07:21.236076 Epoch 80 , Training loss 0.2552482603461879
2024-04-29 14:07:27.368557 Epoch 90 , Training loss 0.2437810770645263
2024-04-29 14:07:33.519440 Epoch 100 , Training loss 0.23065657842500953
